## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import tensorflow as tf

In [3]:
# Create simple network with Keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [5]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [6]:
#from google.colab import drive
#drive.mount('/gdrive')

In [39]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [40]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [41]:
data.drop(['date','symbol'],axis=1,inplace=True)

In [42]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [43]:
data.shape

(851264, 5)

In [44]:
data = data[0:1000]

In [45]:
data.shape

(1000, 5)

In [46]:
data[['volume']] = data[['volume']] / 1000000

### Divide the data into train and test sets

In [47]:
y=data['volume']
x=data.iloc[:,0:4]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x.values, y.values, test_size=0.3,random_state=456)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(700, 4) (700,)
(300, 4) (300,)


In [48]:
#X = data.drop('volume', axis=1)  
#y = data['volume']

In [49]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)  

In [50]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(700, 4)
(300, 4)
(700,)
(300,)


#### Convert Training and Test Data to numpy float32 arrays


In [51]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_train=y_train.astype('float32')
y_test=y_test.astype('float32')

In [52]:
#X_train = np.array(X_train[:], dtype=np.float)

In [53]:
#X_test = np.array(X_test[:], dtype=np.float)

In [54]:
#y_train = np.array(y_train[:], dtype=np.float)

In [55]:
#y_test = np.array(y_test[:], dtype=np.float)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [56]:
from sklearn.preprocessing import Normalizer
X_train = Normalizer().fit_transform(X_train)
X_test = Normalizer().transform(X_test)

In [57]:
#X_train = Normalizer().fit(X_train)

In [58]:
#X_train

In [59]:
#X_test = Normalizer().fit(X_test)

In [60]:
#X_test

In [61]:
print('Number of examples: ', X_train.shape[0])
print('Number of features for each example: ', X_train.shape[1])
print('Shape of actual prices data: ', y_train.shape)

Number of examples:  700
Number of features for each example:  4
Shape of actual prices data:  (700,)


In [62]:
print('Number of examples: ', X_test.shape[0])
print('Number of features for each example: ', X_test.shape[1])
print('Shape of actual prices data: ', y_test.shape)

Number of examples:  300
Number of features for each example:  4
Shape of actual prices data:  (300,)


## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [63]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

2.Define a function to calculate prediction

In [64]:
class Model():
      def __init__(self):
        self.W = W
        self.B = b 
        self.variables = [ self.W, self.B ]
    
      def frwrd_pass(self,X_train):
        out = tf.matmul( X_train, self.W ) + self.B
        return out

3.Loss (Cost) Function [Mean square error]

In [65]:
#loss = tf.reduce_mean(tf.square(y-y_train),name='Loss')
def loss(predicted_y, desired_y):
      return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [66]:
#train_op = tf.train.GradientDescentOptimizer(0.03).minimize(Loss_1)

In [67]:
def train(X_train, Y_train, epochs,test):
    print(test)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.05)
    for i in range(epochs):
        with tf.GradientTape() as tape:
            predicted = model.frwrd_pass(X_train)
            curr_loss = loss(predicted, Y_train)
            grads = tape.gradient( curr_loss, model.variables )
            optimizer.apply_gradients(zip(grads, model.variables),global_step=tf.train.get_or_create_global_step())
            if test == 0 :
                print("Loss at step {:d}: {:.3f}".format(i, loss(model.frwrd_pass( X_train ), Y_train)))
            if test == 1 :
                if i % 5 == 0:
                     print("Loss at step {:d}: {:.3f}".format(i, loss(model.frwrd_pass( X_train ), Y_train)))

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [68]:
#how many times data need to be shown to model
model = Model()
training_epochs = 100
train(X_train,y_train,training_epochs,0)

0
Loss at step 0: 175.769
Loss at step 1: 169.423
Loss at step 2: 165.361
Loss at step 3: 162.761
Loss at step 4: 161.097
Loss at step 5: 160.032
Loss at step 6: 159.351
Loss at step 7: 158.915
Loss at step 8: 158.635
Loss at step 9: 158.457
Loss at step 10: 158.342
Loss at step 11: 158.269
Loss at step 12: 158.222
Loss at step 13: 158.193
Loss at step 14: 158.173
Loss at step 15: 158.161
Loss at step 16: 158.153
Loss at step 17: 158.148
Loss at step 18: 158.145
Loss at step 19: 158.143
Loss at step 20: 158.141
Loss at step 21: 158.141
Loss at step 22: 158.140
Loss at step 23: 158.140
Loss at step 24: 158.140
Loss at step 25: 158.139
Loss at step 26: 158.139
Loss at step 27: 158.139
Loss at step 28: 158.139
Loss at step 29: 158.139
Loss at step 30: 158.139
Loss at step 31: 158.139
Loss at step 32: 158.139
Loss at step 33: 158.139
Loss at step 34: 158.139
Loss at step 35: 158.139
Loss at step 36: 158.139
Loss at step 37: 158.139
Loss at step 38: 158.139
Loss at step 39: 158.139
Loss at 

In [69]:
train(X_test,y_test,training_epochs,1)

1
Loss at step 0: 327.825
Loss at step 5: 327.797
Loss at step 10: 327.795
Loss at step 15: 327.794
Loss at step 20: 327.794
Loss at step 25: 327.794
Loss at step 30: 327.794
Loss at step 35: 327.794
Loss at step 40: 327.794
Loss at step 45: 327.794
Loss at step 50: 327.794
Loss at step 55: 327.794
Loss at step 60: 327.794
Loss at step 65: 327.794
Loss at step 70: 327.794
Loss at step 75: 327.794
Loss at step 80: 327.794
Loss at step 85: 327.794
Loss at step 90: 327.794
Loss at step 95: 327.794


### Get the shapes and values of W and b

In [70]:
print('W:',W)
print('b:',b)

W: <tf.Variable 'Weights:0' shape=(4, 1) dtype=float32, numpy=
array([[1.365027 ],
       [1.3696388],
       [1.3510374],
       [1.381682 ]], dtype=float32)>
b: <tf.Variable 'Bias:0' shape=(1,) dtype=float32, numpy=array([2.7338476], dtype=float32)>


In [71]:
y_test_pred = tf.add(tf.matmul(X_test,W),b,name='output')

In [72]:
loss(y_test_pred, y_test)

<tf.Tensor: id=9584, shape=(), dtype=float32, numpy=327.79434>

### Model Prediction on 1st Examples in Test Dataset

In [73]:
y_test_pred[0,:]

<tf.Tensor: id=9589, shape=(1,), dtype=float32, numpy=array([5.4676213], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

Note   

Restart the Kernel to compute below  codes

In [13]:
#assert not tf.executing_eagerly()

In [1]:
import tensorflow as tf

In [2]:
# Create simple network with Keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
import pandas as pd
df = pd.read_csv('Iris.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [4]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [5]:
df=pd.get_dummies(df)
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [6]:
x = df.iloc[:,1:5]
y = df.iloc[:,5:8]

### Divide the dataset into Training and test (70:30)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3,random_state=456)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(105, 4) (105, 3)
(45, 4) (45, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [8]:
# Create simple network with Keras
from keras.models import Sequential
from keras.layers import Dense

tf.disable_eager_execution()
# Extending the first model with activation functions
model = Sequential()
#specifying activation functions
model.add(Dense(3, input_dim=4, init='uniform', activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\admin\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, input_dim=4, activation="softmax", kernel_initializer="uniform")`
  if __name__ == '__main__':


### Model Training 

In [9]:
# Fit the model
model.fit(X_train, y_train, nb_epoch=20, batch_size=10,  verbose=2)
# evaluate the model
scores = model.evaluate(X_train, y_train)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Instructions for updating:
Use tf.cast instead.


C:\Users\admin\Anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
 - 0s - loss: 1.0972 - acc: 0.3429
Epoch 2/20
 - 0s - loss: 0.9931 - acc: 0.6286
Epoch 3/20
 - 0s - loss: 0.9133 - acc: 0.7143
Epoch 4/20
 - 0s - loss: 0.8651 - acc: 0.7143
Epoch 5/20
 - 0s - loss: 0.8275 - acc: 0.7143
Epoch 6/20
 - 0s - loss: 0.7781 - acc: 0.7238
Epoch 7/20
 - 0s - loss: 0.7422 - acc: 0.7143
Epoch 8/20
 - 0s - loss: 0.7213 - acc: 0.7238
Epoch 9/20
 - 0s - loss: 0.7012 - acc: 0.7238
Epoch 10/20
 - 0s - loss: 0.6718 - acc: 0.7333
Epoch 11/20
 - 0s - loss: 0.6558 - acc: 0.7810
Epoch 12/20
 - 0s - loss: 0.6357 - acc: 0.7143
Epoch 13/20
 - 0s - loss: 0.6137 - acc: 0.7429
Epoch 14/20
 - 0s - loss: 0.6019 - acc: 0.7143
Epoch 15/20
 - 0s - loss: 0.5868 - acc: 0.7714
Epoch 16/20
 - 0s - loss: 0.5776 - acc: 0.7143
Epoch 17/20
 - 0s - loss: 0.5651 - acc: 0.7238
Epoch 18/20
 - 0s - loss: 0.5569 - acc: 0.7143
Epoch 19/20
 - 0s - loss: 0.5510 - acc: 0.7714
Epoch 20/20
 - 0s - loss: 0.5419 - acc: 0.7238
105/105 [==============================] - 0s 761us/step
acc: 73.33%


### Model Prediction

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


### Save the Model

In [11]:
scores = model.evaluate(X_test, y_test)
print(scores)
pred = model.predict(X_test)
pred

45/45 [==============================] - 0s 111us/step
[0.6397718985875448, 0.5555555635028415]


array([[0.0466155 , 0.3173738 , 0.63601065],
       [0.13468789, 0.37581503, 0.48949713],
       [0.7428204 , 0.177284  , 0.07989564],
       [0.17225806, 0.3717687 , 0.45597324],
       [0.77486503, 0.15579969, 0.06933536],
       [0.05590733, 0.34418064, 0.5999121 ],
       [0.03364322, 0.32241106, 0.6439457 ],
       [0.23418888, 0.37234083, 0.39347026],
       [0.06030503, 0.3214496 , 0.61824536],
       [0.15185875, 0.36702555, 0.4811157 ],
       [0.10796031, 0.37222525, 0.51981443],
       [0.15998136, 0.3918509 , 0.4481677 ],
       [0.72844934, 0.18251126, 0.08903936],
       [0.10974037, 0.38043347, 0.5098261 ],
       [0.0645819 , 0.3403075 , 0.5951106 ],
       [0.05175395, 0.32236084, 0.6258852 ],
       [0.04703782, 0.29806125, 0.65490097],
       [0.16005687, 0.38916376, 0.4507794 ],
       [0.8071229 , 0.1373106 , 0.0555665 ],
       [0.12417835, 0.37425327, 0.5015684 ],
       [0.02917464, 0.27099022, 0.6998351 ],
       [0.13203807, 0.3851076 , 0.48285428],
       [0.

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [12]:
model.save('iris.h5')